# Training Deep Neural Networks

## Practical 2

_**Experiment on Fashion MNIST or any other appropriate dataset to check if reusing pretrained layers in transfer learning makes the training possible with less data and saves training time. Also check for any model performance improvement.**_

First a model will be trained on set A (for classification task with 8 classes such as trousers, dresses, coats, sandals, shirts, sneakers, bags, and ankle boots in Fashion MNIST dataset). Then the learning will be reused for another a binary classification task on set B (the remaining 2 classes such as T-shirts/tops and pullovers in the saame dataset) since classes in set A are somewhat similar to classes in set B.

In [59]:
# Imports required packages

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split

In [60]:
# Loads fashion mnist dataset
fashion = fashion_mnist.load_data()

In [61]:
# Considering dataset is organized in tuple, items are referenced as follows
(X_train_full, y_train_full), (X_test, y_test) = fashion

In [62]:
# Checks the shape of the datasets
print("Train dataset shape:", X_train_full.shape,
      "\nTest dataset shape:", X_test.shape)

Train dataset shape: (60000, 28, 28) 
Test dataset shape: (10000, 28, 28)


In [63]:
# Each training and test example is assigned to one of the following labels.
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", \
               "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# Normalizes the data between 0 and 1 for effective neural network model training
X_train_full, X_test = X_train_full / 255., X_test / 255.

In [64]:
# Splits train dataset further to seperate 5000 instances to be used as validation set

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=5000, random_state=42, stratify=y_train_full)

**Splits Fashion MNIST into tasks A and B, then train and save.**

In [66]:
# Stores class ids of the target items into variables.
# In this case, variable pos_class_id and neg_class_id will store 2 and 0, respectively.
# Also, item "Pullover" and "T-shirt/top" are considered as positive and negative class, respectively.

pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

In [67]:
def split_dataset(X, y):
    """
    Splits the dataset having all items into a pair of tuples - one for dataset for 8-class classification task
    and other one for dataset for the remaining 2-class classification task.
    """
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
        
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

In [68]:
# Splits train, validation and test data into respective dataset for classification task A and B

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_val_A, y_val_A), (X_val_B, y_val_B) = split_dataset(X_val, y_val)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)

# Considers only 200 instances for training for classification task B
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [69]:
# Creates a dense nueral network for classification task A

tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])

In [70]:
# Fits the model.
history = model_A.fit(X_train_A, y_train_A, epochs=20, validation_data=(X_val_A, y_val_A))

Epoch 1/20
1375/1375 [==============================] - 6s 4ms/step - loss: 1.1326 - accuracy: 0.6558 - val_loss: 0.7083 - val_accuracy: 0.7832
Epoch 2/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.5987 - accuracy: 0.8101 - val_loss: 0.5215 - val_accuracy: 0.8322
Epoch 3/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.4805 - accuracy: 0.8459 - val_loss: 0.4399 - val_accuracy: 0.8633
Epoch 4/20
1375/1375 [==============================] - 9s 7ms/step - loss: 0.4224 - accuracy: 0.8627 - val_loss: 0.3907 - val_accuracy: 0.8775
Epoch 5/20
1375/1375 [==============================] - 12s 9ms/step - loss: 0.3863 - accuracy: 0.8726 - val_loss: 0.3696 - val_accuracy: 0.8825
Epoch 6/20
1375/1375 [==============================] - 13s 9ms/step - loss: 0.3609 - accuracy: 0.8797 - val_loss: 0.3438 - val_accuracy: 0.8903
Epoch 7/20
1375/1375 [==============================] - 9s 6ms/step - loss: 0.3424 - accuracy: 0.8847 - val_loss: 0.3216 - val_accurac

In [71]:
# Saves the model to be used later for transfer learning
# NOTE: Make sure the folder "models" exists under the current working directory

model_A.save("./models/my_fashion_mnist_model_A.keras")

INFO:tensorflow:Assets written to: ./models/my_fashion_mnist_model_A/assets


INFO:tensorflow:Assets written to: ./models/my_fashion_mnist_model_A/assets


Now, to realize whether if transfer learning works or not, first train model B, then evaluate it, without reusing model A

In [73]:
# Creates a dense nueral network for classification task B

tf.random.set_seed(42)

model_B = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_B.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])

In [74]:
# Fits the model.
history = model_B.fit(X_train_B, y_train_B, epochs=20, validation_data=(X_val_B, y_val_B))

Epoch 1/20
7/7 [==============================] - 1s 76ms/step - loss: 0.7406 - accuracy: 0.4150 - val_loss: 0.6996 - val_accuracy: 0.4610
Epoch 2/20
7/7 [==============================] - 0s 24ms/step - loss: 0.7003 - accuracy: 0.4350 - val_loss: 0.6684 - val_accuracy: 0.5350
Epoch 3/20
7/7 [==============================] - 0s 20ms/step - loss: 0.6688 - accuracy: 0.5600 - val_loss: 0.6431 - val_accuracy: 0.7030
Epoch 4/20
7/7 [==============================] - 0s 25ms/step - loss: 0.6429 - accuracy: 0.6850 - val_loss: 0.6235 - val_accuracy: 0.7810
Epoch 5/20
7/7 [==============================] - 0s 25ms/step - loss: 0.6230 - accuracy: 0.7600 - val_loss: 0.6069 - val_accuracy: 0.8150
Epoch 6/20
7/7 [==============================] - 0s 29ms/step - loss: 0.6064 - accuracy: 0.8000 - val_loss: 0.5911 - val_accuracy: 0.8390
Epoch 7/20
7/7 [==============================] - 0s 22ms/step - loss: 0.5903 - accuracy: 0.8200 - val_loss: 0.5772 - val_accuracy: 0.8440
Epoch 8/20
7/7 [===========

In [75]:
# Evaluates the model on test dataset
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.4546 - accuracy: 0.9205


[0.45464184880256653, 0.9204999804496765]

Model B reaches 92.04% accuracy on the test set. Now let's try reusing the pretrained model A.

In [77]:
# Loads the model trained for classification task A
model_A = tf.keras.models.load_model("./models/my_fashion_mnist_model_A.keras")

# Creates a new model copies all the layers, except for the output layer, from model A
# to be reused for classification task B
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])

# Just one-node output layer is added into the new model
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

As model_B_on_A and model_A actually share layers now, training one will update both models. To avoid that model_B_on_A should be built on top of a clone of model_A.

In [79]:
tf.random.set_seed(42)

# Network architecture of model A is cloned into a new model
model_A_clone = tf.keras.models.clone_model(model_A)

# Learned weigts of the model A are copied into the new cloned model
model_A_clone.set_weights(model_A.get_weights())

In [80]:
# Creates model_B_on_A just like in the previous cell

# Like the previous steps, creates target model B cloning all layers, except for the output layer,
# and then adds a one-node output layer onto it

model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

**Target model's training takes place into two phases. In the first phase, only output layer gets trained over a shorter iterations keeping all hidden layers non-trainable, and in the second phase, all layers are trained over a relatively longer iterations.**

In [82]:
# Freezes all the hidden layers before training
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [83]:
# Fits the model over a shorter iteration to train only the output layer
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_val_B, y_val_B))

Epoch 1/4
7/7 [==============================] - 1s 98ms/step - loss: 0.8908 - accuracy: 0.5250 - val_loss: 0.7991 - val_accuracy: 0.5050
Epoch 2/4
7/7 [==============================] - 0s 22ms/step - loss: 0.7271 - accuracy: 0.5750 - val_loss: 0.7215 - val_accuracy: 0.5580
Epoch 3/4
7/7 [==============================] - 0s 21ms/step - loss: 0.6843 - accuracy: 0.6050 - val_loss: 0.6976 - val_accuracy: 0.5800
Epoch 4/4
7/7 [==============================] - 0s 40ms/step - loss: 0.6639 - accuracy: 0.6650 - val_loss: 0.6678 - val_accuracy: 0.6320


In [84]:
# Then allows hidden layers trainable before next iterations of training
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [85]:
# Fits the full model.
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_val_B, y_val_B))

Epoch 1/16
7/7 [==============================] - 1s 52ms/step - loss: 0.6206 - accuracy: 0.6900 - val_loss: 0.5894 - val_accuracy: 0.7440
Epoch 2/16
7/7 [==============================] - 0s 23ms/step - loss: 0.5528 - accuracy: 0.8100 - val_loss: 0.5388 - val_accuracy: 0.8050
Epoch 3/16
7/7 [==============================] - 0s 22ms/step - loss: 0.5014 - accuracy: 0.8600 - val_loss: 0.5111 - val_accuracy: 0.8100
Epoch 4/16
7/7 [==============================] - 0s 47ms/step - loss: 0.4588 - accuracy: 0.8650 - val_loss: 0.4579 - val_accuracy: 0.8860
Epoch 5/16
7/7 [==============================] - 0s 23ms/step - loss: 0.4233 - accuracy: 0.9000 - val_loss: 0.4297 - val_accuracy: 0.8950
Epoch 6/16
7/7 [==============================] - 0s 25ms/step - loss: 0.3972 - accuracy: 0.9150 - val_loss: 0.4025 - val_accuracy: 0.9110
Epoch 7/16
7/7 [==============================] - 0s 25ms/step - loss: 0.3678 - accuracy: 0.9350 - val_loss: 0.3990 - val_accuracy: 0.8950
Epoch 8/16
7/7 [===========

In [86]:
# Once again, evaluates the model B against test dataset
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 4ms/step - loss: 0.2919 - accuracy: 0.9270


[0.2919308841228485, 0.9269999861717224]

The evaluation shows that there is 0.65% [92.69 - 92.04] improvement in accuracy and it indicates that error drop rate is 8% [1 - (100 - 92.69) / (100 - 92.04)].